In [1]:
import simtk.openmm.app
import simtk.openmm
import simtk.unit as unit
import configparser
import numpy as np
import itertools
import scipy.spatial.distance as sdist
import os
import pdbfixer
import pandas
import subprocess
import nose

In [2]:
import openna

The location is /mnt/d/linux/openna
4.184


In [3]:
import simtk.openmm
pdb=simtk.openmm.app.PDBFile('clean.pdb')
top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField(openna.xml)
s=forcefield.createSystem(top)

In [4]:
rna=openna.RNA.fromCoarsePDB('clean.pdb')
rna.periodic=False
rna.parseConfigurationFile(configuration_file=f'./openna.conf')

In [7]:
import forceTerm
all_forces = dict(Backbone=forceTerm.Backbone,
                 Angle=forceTerm.Angle)

for each_one in all_forces:
    force_now = all_forces[each_one](rna)
    s.addForce(force_now)